In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By

inei_url = 'https://proyectos.inei.gob.pe/iinei/srienaho/index.htm'
driver = webdriver.Edge()
driver.get(inei_url)

In [2]:
driver.execute_script("Ingresar()")

In [3]:
# links
from bs4 import BeautifulSoup
from io import StringIO
import pandas as pd
import time

def get_table_info_mod(driver):

    table = driver.find_element(
        By.ID, "divDetalle"
    )
    table = table.find_element(By.TAG_NAME, 'table')

    cells_in_table = table.find_elements(By.XPATH, './/td/a')

    links = []
    for cell in cells_in_table:
        href = cell.get_attribute('href')
        if href:
            links.append(href)


    table_html = driver.find_element(By.ID, "divDetalle").get_attribute("outerHTML")

    soup = BeautifulSoup(table_html, 'html.parser')
    table = soup.find('table')
    df = pd.read_html(StringIO(str(table)))[0]
    len(df)

    import numpy as np, re
    array = np.array(links)

    df1 = pd.DataFrame(array.reshape(-1, int(len(links) / len(df))))

    keywords = ["VerPreguntas", "VerFicha", "SPSS", "STATA", "CSV", "DBF"]

    # Función para limpiar texto y extraer solo las palabras clave relevantes
    def extract_keywords(column):
        # Buscar palabras clave en cada fila de la columna
        return column.apply(lambda x: ' '.join(
            word for word in keywords if word in x
        ))

    # Procesar el DataFrame para extraer las palabras clave relevantes
    processed_df = df1.apply(extract_keywords)

    # Función para encontrar el texto más repetido en cada columna
    def most_frequent_text(column):
        return column.mode()[0] if not column.mode().empty else None

    # Generar el array con el texto más repetido por columna
    result = processed_df.apply(most_frequent_text).tolist()

    df1.columns = result

    result_df = pd.concat((df, df1), axis = 1)

    return result_df

# get_table_info_mod(driver)

In [4]:
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

In [5]:
driver.find_element(By.XPATH, '/html/body/form/center/table[2]/tbody/tr[1]/td[2]/div[1]/span/span[1]/span').click()
time.sleep(1)
actions = ActionChains(driver)

# Simular la acción de presionar la flecha hacia abajo 3 veces y luego Enter
actions.send_keys(Keys.ARROW_DOWN)\
       .send_keys(Keys.ARROW_DOWN)\
       .send_keys(Keys.ENTER)\
       .perform()
Select(driver.find_element(By.NAME, 'cmbEncuestaN')).select_by_index(1)
time.sleep(.3)

In [6]:
actions.send_keys(Keys.TAB).send_keys(Keys.DOWN).perform()

In [7]:
cmb_anio = Select(driver.find_element(By.NAME, "cmbAnno"))
data = []
for anio in cmb_anio.options:
    year_value = anio.get_attribute('value')
    if year_value.strip() == "":
        continue
    time.sleep(0.5)
    cmb_periodo = Select(driver.find_element(By.NAME, "cmbTrimestre"))
    cmb_periodo.select_by_index(1)
    time.sleep(0.5)
    data_i = get_table_info_mod(driver)
    cmb_anio.select_by_value(year_value)
    data.append(data_i)


In [11]:
metadata_inei = pd.concat(data)#.drop_duplicates()
metadata_inei.to_csv("../../MetadataSources/inei/enaho_actualizado.csv", index=False)
